# Read and process AIS data

- Read AIS (ship location) data from the Danish costguard
- See ship traffic for a day
- Locate any ships sailing into the area between the offshore turbines

In [ ]:
# Standard library
from pathlib import Path
import zipfile
import datetime as dt
import json

# Data analysis
import pandas as pd

# Geospatial libraries
import geopandas as gpd
import pydeck as pdk


In [ ]:
if not Path("aisdk-2023-01.zip").exists():
    raise Exception("Unable to find AIS data. Please download it from http://web.ais.dk/aisdata/")

In [ ]:
# Read all ship location information for one day
with zipfile.ZipFile("aisdk-2023-01.zip") as z:
    # Load only the first day
    with z.open("aisdk-2023-01-01.csv") as f:
        ships = pd.read_csv(f)

In [ ]:
# Remove ships that don't transmit a callsign
ships.dropna(subset=('Callsign'), inplace=True)

# Clean up raw data
ships.sort_values(by=['Callsign', '# Timestamp'], inplace=True)

In [ ]:
# Join all GPS points to a single line for all ships
ship_track = []
for g in ships.groupby('Callsign')[['Longitude', 'Latitude']]:
    path = g[1]
    path = path.loc[path['Latitude'] < 70, :]
    path = path.to_numpy()[::10].tolist()
    if len(path) > 10 and g[0] != "0":
        ship_track.append({"Callsign": str(g[0]), 'path': path})
    
df_ship_track = pd.DataFrame.from_dict(ship_track)

In [ ]:
# Load wind turbine offshore locations
json = pd.read_json('../../NoCode/data/dk_offshore_sites.geojson')
sites = pd.DataFrame()

# Parse the geometry out in Pandas
sites["coordinates"] = json["features"].apply(lambda row: row["geometry"]["coordinates"])

In [ ]:
# Make map
view_state = pdk.ViewState(latitude=55.782556, longitude=11.3484867, zoom=5)

# Draw ship routes as paths
layer = pdk.Layer(
    type="PathLayer",
    data=df_ship_track,
    pickable=True,
    width_scale=20,
    get_color=(255,255,255),
    width_min_pixels=2,
    get_path="path",
    get_width=5,
)

# Draw turbine locations as polygons
layer_turbines = pdk.Layer(
    type="PolygonLayer",
    data=sites,
    pickable=True,
    get_fill_color=[178,34,34],
    get_line_color=[255, 255, 255],
    get_polygon="coordinates",
    opacity=0.8,
    auto_highlight=True,
)

r = pdk.Deck(layers=[layer, layer_turbines], initial_view_state=view_state, tooltip={"text": "{Callsign}"})
r.to_html("path_layer.html")

## Extract all ships that sails into any wind park area

In [ ]:
# Read all ship location information of each available day
possible_supply_ships = []
with zipfile.ZipFile("aisdk-2023-01.zip") as z:
    for fobj in z.filelist:
        print("Reading file", fobj.filename)

        # Load only the first day
        with z.open(fobj) as f:
            ships_daily = pd.read_csv(f)

            # Remove ships that don't transmit a callsign and ships that transmit strange positions
            ships_daily.dropna(subset=('Callsign'), inplace=True)
            ships_daily['Callsign'] = ships_daily['Callsign'].astype(str)
            ships_daily = ships_daily[ships_daily["Callsign"] != "0"]
            
            # Remove points that are clearly wrong - Denmark doesn't exist above 60 degrees north
            ships_daily = ships_daily[ships_daily["Latitude"]<60]

            # Find ships that travel into any of the wind park areas using a spatial join
            geo_ships = gpd.GeoDataFrame(ships_daily, 
                                         geometry=gpd.points_from_xy(
                                             ships_daily.Longitude,
                                             ships_daily.Latitude),
                                         crs="EPSG:4326")

            # Clean up raw data
            ships.sort_values(by=['Callsign', '# Timestamp'], inplace=True)

            # Find all ship locations within the wind parks
            points_within_site = geo_sites\
                                .sjoin(geo_ships, how='left')\
                                .dropna(subset='MMSI')

            ship_names = "|".join(points_within_site.Callsign.unique())
            print("Ships within area", ship_names)

            # Only keep locations from the ships that passed the areas this day
            ship_within_area = ships_daily[ships_daily["Callsign"].str.contains(ship_names, regex=True)]
            
            # Save for later
            possible_supply_ships.append(ship_within_area)
            
know_ships = pd.concat(possible_supply_ships)

In [ ]:
features = []
know_ships['# Timestamp'] = pd.to_datetime(know_ships['# Timestamp'], format="%d/%m/%Y %H:%M:%S")

for key, ship in know_ships.groupby('Callsign'):
    epoch = (ship['# Timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds().astype(int)
    coords = pd.concat([ship['Longitude'], ship['Latitude'], epoch], axis=1)
    coords.insert(loc=2, column='z', value=0)
    
    f = {
        "type": "Feature",
        "properties": {
            "callsign": key
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coords.to_numpy().tolist()
        }
    }
    features.append(f)
    
geojson = {
    "type": "FeatureCollection",
    "features": features
}

In [ ]:
json.dump(geojson, open('ships.json', 'w'))

In [ ]:
def dataframe_to_timeline(df, resample=1):
    features = []
    df['# Timestamp'] = pd.to_datetime(df['# Timestamp'], format="%d/%m/%Y %H:%M:%S")

    for callsign, ship in df.groupby('Callsign'):
        if callsign == "0":
            # Callsign 0 isn't a ship
            continue

        # Resample to every N samples
        ship = ship.loc[::resample, :]

        # Remove duplicates GPS locations (e.g. when ships are docked)
        ship = ship.drop_duplicates(subset=["Latitude", "Longitude"])
        ship = ship[ship["Longitude"] > 0]

        # Convert to format for rendering
        epoch = (ship['# Timestamp'] - dt.datetime(1970,1,1)).dt.total_seconds().astype(int)
        coords = pd.concat([ship['Longitude'], ship['Latitude'], epoch], axis=1)
        coords.insert(loc=2, column='z', value=0)

        f = {
            "type": "Feature",
            "properties": {
                "callsign": callsign
            },
            "geometry": {
                "type": "LineString",
                "coordinates": coords.to_numpy().tolist()
            }
        }
        features.append(f)

    return {
        "type": "FeatureCollection",
        "features": features
    }

In [ ]:
json_day_one = dataframe_to_timeline(ships, 20)

In [ ]:
json.dump(json_day_one, open('ships_Jan_first_2023.json', 'w'))